In [1]:
import numpy as np
import pandas as pd
import json
import os

In [2]:
# Parameters
N = 2
a = 1.0
c = -0.2

potential = 'quadratic'
boundary_condition = 'periodic'
cutoff = 4

In [3]:
folder_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\Wess-Zumino\VQE\DE\Optimizing\VaryStrategy\periodic\quadratic\C0.2\N2"
dataframes = {}
folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
folders

['2025-02-10_14-11-28',
 '2025-02-10_16-35-39',
 '2025-02-10_16-36-22',
 '2025-02-10_16-37-01',
 '2025-02-10_16-37-45',
 '2025-02-10_16-38-20',
 '2025-02-10_17-03-40',
 '2025-02-10_17-04-03',
 '2025-02-10_17-04-26',
 '2025-02-10_17-04-37',
 '2025-02-10_17-04-59',
 '2025-02-10_17-05-11']

In [8]:
converged_runs = []
mean_iters = []
min_e = []
delta_min_e = []
median_e = []
delta_median_e = []
exact_e = []
time_taken = []

num_vqe = []
shots = []
tolerance = []
abs_tolerance = []
strategy = []
popsize = []
maxiter = []

strategies = []


for f in folders:

    data_path = folder_path + "\\{}\\{}_{}.json".format(f,potential, cutoff)
    
    with open(data_path, 'r') as file:
        data = json.load(file)

    strategies.append(data['Optimizer']['strategy'])

    num_vqe.append(data['num_VQE'])
    shots.append(data['shots'])

    optimzer = data['Optimizer']
    maxiter.append(optimzer['maxiter'])
    tolerance.append(optimzer['tolerance'])
    abs_tolerance.append(optimzer['abs_tolerance'])
    strategy.append(optimzer['strategy'])
    popsize.append(optimzer['popsize'])

    #success = sum(data['success'])
    
    exact_min_e = np.min(data['exact_eigenvalues'])
    exact_e.append(exact_min_e)

    converged_runs.append(sum(data['success']))
    mean_iters.append(int(np.round(np.mean(data['num_iters']))))
    min_e.append(np.min(data['results']))
    delta_min_e.append(abs(exact_min_e - np.min(data['results'])))
    median_e.append(np.median(data['results']))
    delta_median_e.append(abs(exact_min_e - np.median(data['results'])))
    
    time_taken.append(data['parallel_run_time'])

    

In [9]:
df = pd.DataFrame({
    'folder': strategies,
    'num_vqe': num_vqe,
    'shots': shots,
    'tolerance': tolerance,
    'abs_tolerance': abs_tolerance,
    'strategy': strategy,
    'popsize': popsize,
    'Converged Runs': [f"{c}/40" for c in converged_runs],
    'maxiter': maxiter,
    'Mean Iter': mean_iters,
    'VQE min E': [f"{num:.5e}" for num in min_e],
    'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    'VQE median E': [f"{num:.5e}" for num in median_e],
    'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    'Exact': [f"{num:.5e}" for num in exact_e],
    'Time': time_taken
})

In [10]:
df.sort_values(['Delta min E', 'Delta median E'], ascending=[False,False])

,folder,num_vqe,shots,tolerance,abs_tolerance,strategy,popsize,Converged Runs,maxiter,Mean Iter,VQE min E,Delta min E,VQE median E,Delta median E,Exact,Time
6,rand2exp,40,1024,0.001,0.001,rand2exp,20,0/40,10000,10000,4.11547e-01,5.33728e-03,4.20650e-01,1.44403e-02,4.06210e-01,0:24:18.625967
4,rand1exp,40,1024,0.001,0.001,rand1exp,20,0/40,10000,10000,4.08878e-01,2.66864e-03,4.18031e-01,1.18212e-02,4.06210e-01,0:24:18.147869
10,best2exp,40,1024,0.001,0.001,best2exp,20,0/40,10000,10000,4.06210e-01,1.88738e-15,4.20949e-01,1.47393e-02,4.06210e-01,0:24:34.340174
5,rand2bin,40,1024,0.001,0.001,rand2bin,20,0/40,10000,10000,4.24826e-01,1.86164e-02,5.81211e-01,1.75001e-01,4.06210e-01,0:24:25.586108
9,currenttobest1exp,40,1024,0.001,0.001,currenttobest1exp,20,0/40,10000,10000,4.06210e-01,1.77636e-15,4.13430e-01,7.22013e-03,4.06210e-01,0:24:21.172800
8,currenttobest1bin,40,1024,0.001,0.001,currenttobest1bin,20,27/40,10000,5323,4.06210e-01,1.77636e-15,4.06877e-01,6.67160e-04,4.06210e-01,0:19:46.305118
1,best1bin,40,1024,0.001,0.001,best1bin,20,16/40,10000,6903,4.06210e-01,1.77636e-15,4.12881e-01,6.67160e-03,4.06210e-01,0:21:50.563143
7,randtobest1exp,40,1024,0.001,0.001,randtobest1exp,20,8/40,10000,9426,4.06210e-01,1.77636e-15,4.06210e-01,2.38698e-15,4.06210e-01,0:23:58.021870
3,rand1bin,40,1024,0.001,0.001,rand1bin,20,32/40,10000,6414,4.06210e-01,1.77636e-15,4.06210e-01,2.10942e-15,4.06210e-01,0:20:38.454896
0,randtobest1bin,40,1024,0.001,0.001,randtobest1bin,20,24/40,10000,5583,4.06210e-01,1.77636e-15,4.07544e-01,1.33432e-03,4.06210e-01,0:20:06.392612
